In [1]:
import scipy.sparse.linalg as l
import numpy as np

def hooi(A,dim,n): 
    (x,y,z)=A.shape
    #print("A "+str(A.shape))
    (r1,r2,r3) = dim

    R_init = np.random.rand(y,y)
    R,w,v = l.svds(R_init,r2)
    #print("R "+str(R.shape))
    
    V_init = np.random.rand(z,z)
    V,w,v = l.svds(V_init,r3)
    #print("V "+str(V.shape))

    for i in range(n):
        print(i,end=" ")
        # Calculate L estimate
        C2 = np.tensordot(A,R,axes=(1,0))
        #print("C2 "+str(C2.shape))
        C1 = np.tensordot(C2,V,axes=(1,0))
        #print("C1 "+str(C1.shape))
        C = C1.reshape(x,r2*r3)
        L,w,v = l.svds(C,r1)
        #print("L "+str(L.shape))
    
        # Calculate R estimate
        D2 = np.tensordot(A,L,axes=(0,0))
        #print("D2 "+str(D2.shape))
        D1 = np.tensordot(D2,V,axes=(1,0))
        #print("D1 "+str(D1.shape))
        D = D1.reshape(y,r1*r3)
        #print("D "+str(D.shape))
        R,w,v = l.svds(D,r2)
        #print("R "+str(R.shape))
    
        # Calculate V estimate
        E2 = np.tensordot(A,L,axes=(0,0))
        E1 = np.tensordot(E2,R,axes=(0,0))
        #print("E1 "+str(E1.shape))
        E = E1.reshape(z,r1*r2)
        V,w,v=l.svds(E,r3)

    # Calculate the core tensor
    B = np.tensordot(E1,V,axes=(0,0))
    
    # Calculate the convergence
    M2 = np.tensordot(B,L,axes=(0,1))
    M1 = np.tensordot(M2,R,axes=(0,1))
    M = np.tensordot(M1,V,axes=(0,1))
    AA = (A-M)*(A-M)
    a=sum(sum(sum(AA)))
    model = (B,L,R,V)
    converge = (a,M)
    
    return model,converge, C